In [1]:
from Bio import PDB
from Bio.PDB import Superimposer
from Bio.PDB import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from pathlib import Path
import os
import csv
import pandas as pd
from scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
def process_directory(directory):
    for dirpath, dirname, filenames in os.walk(directory):
        batch_number = os.path.basename(dirpath)
        for filename in filenames:
            if filename.endswith(".cif"):
                continue

In [2]:
def calculate_rmsd(fixed_atoms, moving_atoms, structure):
    super_imposer = Superimposer()
    super_imposer.set_atoms(fixed_atoms, moving_atoms)
    super_imposer.apply(structure.get_atoms())
    return super_imposer.rms

In [3]:
output_file = '/home/max/stayahead/snellius/outputs/alpha_var.txt'

In [4]:
p = PDB.PDBParser(QUIET=True)
struct_path = Path('/home/max/stayahead/snellius2/outputs/colabfold/SARS-CoV-ACE2_residues_unrelaxed_rank_001_alphafold2_numcycle_2.pdb')
structure = p.get_structure('fixed', struct_path)
parent_dir = struct_path.parent
key_name = parent_dir.name
print(key_name)
crys_struct = p.get_structure('crystal', '/home/max/stayahead/snellius2/outputs/af/ranked_0.pdb')
crys_model = crys_struct[0]
for chain in crys_model:
    if chain.id == 'E':
        fixed_crys_atoms = [residue['CA'] for residue in chain.get_residues() if 'CA' in residue]
        moving_ref_atoms = [residue['CA'] for residue in structure.get_residues() if 'CA' in residue]
        print(len(fixed_crys_atoms))
        print(len(moving_ref_atoms))
        dif = len(fixed_crys_atoms) - len(moving_ref_atoms)
        print(dif)
                # Adjust the length of moving_ref_atoms based on dif
        if dif < 0:
            # If dif is negative, remove the last -dif atoms from moving_ref_atoms
            moving_ref_atoms = moving_ref_atoms[:dif]
        elif dif > 0:
            # If dif is positive, you might need to address how to handle this case,
            # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
            pass
        # moving_ref_atoms.pop()
        assert len(fixed_crys_atoms) == len(moving_ref_atoms)
        rmsd = calculate_rmsd(fixed_crys_atoms, moving_ref_atoms, chain)
        # with open(output_file, 'w') as f:
        #     f.write(f'{rmsd}\n')
        print(rmsd)


colabfold


In [13]:
p = PDB.PDBParser(QUIET=True)
struct_path = Path('/home/max/stayahead/snellius2/outputs/colabfold/SARS-CoV-ACE2_residues_unrelaxed_rank_001_alphafold2_numcycle_1.pdb')
structure = p.get_structure('moving', struct_path)
parent_dir = struct_path.parent
key_name = parent_dir.name
print(key_name)
ref_struct = p.get_structure('fixed', '/home/max/stayahead/snellius2/outputs/af/ranked_0.pdb')
moving_ref_atoms = [residue['CA'] for residue in structure.get_residues() if 'CA' in residue]
fixed_ref_atoms = [residue['CA'] for residue in ref_struct.get_residues() if 'CA' in residue]
print(len(fixed_ref_atoms))
print(len(moving_ref_atoms))
dif = len(fixed_ref_atoms) - len(moving_ref_atoms)
print(dif)
        # Adjust the length of moving_ref_atoms based on dif
if dif < 0:
    # If dif is negative, remove the last -dif atoms from moving_ref_atoms
    moving_ref_atoms = moving_ref_atoms[:dif]
elif dif > 0:
    # If dif is positive, you might need to address how to handle this case,
    # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
    pass
# moving_ref_atoms.pop()
assert len(fixed_ref_atoms) == len(moving_ref_atoms)
rmsd = calculate_rmsd(fixed_ref_atoms, moving_ref_atoms, ref_struct)
print(rmsd)

colabfold
195
195
0
2.8434983230058832


In [4]:
print(len('RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFHHHHHH'))

229


In [6]:
import freesasa
structure = freesasa.Structure('/home/max/stayahead/snellius/outputs/ds4/alpha_variant/ranked_0.pdb')
result = freesasa.calc(structure)
total_sasa = result.totalArea()
# dict[self.n] = total_sasa
print(total_sasa)

10332.772835983595


In [7]:
asdf = str(struct_path)

In [16]:
from Bio import PDB
from Bio.PDB import Superimposer
from Bio.PDB import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from pathlib import Path
import os
import csv
import argparse
# from scipy.spatial import distance
import numpy as np
# import matplotlib.pyplot as plt
import math
from tmtools.io import get_structure, get_residue_data
from tmtools import tm_align
import json
import freesasa

class ComputeMetrics():
    def __init__(self, args):
        self.args = args
        self.input = args.input
        self.output = args.out_dir
        self.ref = args.ref
        # q = query, r = reference, n = name
        self.q, self.r, self.n = self.load_structures()

    
    def load_structures(self):
        p = PDB.PDBParser(QUIET=True)
        struct_path = Path(self.input)
        ref_path = Path(self.ref)
        parent_dir = struct_path.parent
        key_name = parent_dir.name
        fixed_struct = p.get_structure('fixed', ref_path)
        moving_struct = p.get_structure('moving', struct_path)

        return fixed_struct, moving_struct, key_name

    def calculate_rmsd(fixed_atoms, moving_atoms, structure):
        super_imposer = Superimposer()
        super_imposer.set_atoms(fixed_atoms, moving_atoms)
        super_imposer.apply(structure.get_atoms())
        return super_imposer.rms
    
    def rmsd(self, dict):
        fixed_model = self.r[0]
        for chain in fixed_model:
            if chain.id == 'E':
                all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
                all_atoms_moving = [atom for residue in self.q.get_residues() for atom in residue]
                if len(all_atoms_fixed) > len(all_atoms_moving):
                    dif = len(all_atoms_fixed) - len(all_atoms_moving)
                    all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
                else:
                    dif = len(all_atoms_moving) - len(all_atoms_fixed)
                    all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
                    # If dif is positive, you might need to address how to handle this case,
                    # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
                assert len(all_atoms_fixed) == len(all_atoms_moving)
                rmsd = self.calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)
                dict[self.n] = rmsd

        return dict

    def tm_score(self, dict):
        fixed_model = self.r[0]
        for chain in fixed_model:
            if chain.id == 'E':
                all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
                all_atoms_moving = [atom for residue in self.q.get_residues() for atom in residue]
                if len(all_atoms_fixed) > len(all_atoms_moving):
                    dif = len(all_atoms_fixed) - len(all_atoms_moving)
                    all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
                else:
                    dif = len(all_atoms_moving) - len(all_atoms_fixed)
                    all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
                    # If dif is positive, you might need to address how to handle this case,
                    # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
                assert len(all_atoms_fixed) == len(all_atoms_moving)
                tm_score = self.calculate_tm_score(all_atoms_fixed, all_atoms_moving, chain)
                dict[self.n] = tm_score

        return dict
    
    def sasa(self, dict):
        structure = freesasa.Structure(str(self.input))
        result = freesasa.calc(structure)
        total_sasa = result.totalArea()
        dict[self.n] = total_sasa
        # res_dict = {}
        # for i, residue in enumerate(structure.residueAreas()):
        #     residue_sasa = sum(residue.total)
        #     res_dict[i] = residue_sasa
        return dict

In [12]:
def create_parser():
    parser = argparse.ArgumentParser(description="Compute structural metrics.")
    parser.add_argument(
        "-i", "--input", help="Path to input sequence", type=Path, required=True,
    )
    parser.add_argument(
        "-o", "--out_dir", help="Path to save directory for sequences", type=Path, required=True,
    )
    parser.add_argument(
        "-r", "--ref", help="Path to reference structure", type=Path, required=True,
    )
    return parser

parser = create_parser()
args = parser.parse_args('-i /home/max/stayahead/snellius/outputs/ds4/alpha_variant/ranked_0.pdb -o /home/max/stayahead/snellius/outputs/ds4/alpha_variant -r /home/max/stayahead/esm/input/6m0j.pdb'.split())


In [13]:
metrics = ComputeMetrics(args)

In [17]:
rmsd_dict = {}
tm_score_dict = {}
sasa_dict = {}
sasa_dict = metrics.sasa(sasa_dict)
rmsd_dict = metrics.rmsd(rmsd_dict)
tm_score_dict = metrics.tm_score(tm_score_dict)

In [18]:
print(rmsd_dict)

{}
